In [1]:
import os
import pickle
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print(os.listdir("./dataset/"))

['clean_test.csv', 'clean_train.csv', 'sentiment_analysis_test.csv', 'sentiment_analysis_train.csv', 'survey_questions.csv', 'test.csv', 'train.csv']


In [2]:
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RandomizedSearchCV, GridSearchCV, ShuffleSplit
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from wordcloud import WordCloud

# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# create doc2vec vector columns
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint


In [3]:
df_train = pd.read_csv('./dataset/sentiment_analysis_train.csv')
df_test = pd.read_csv('./dataset/sentiment_analysis_test.csv')

In [4]:
df_train.head(3)

,Place,status,clean_positives,clean_negatives,clean_reviews,positive_score,positive_label,negative_score,negative_label,review_score,review_label
0,startup_1,Current Employee,people smart friendly,bureaucracy slow thing,people smart friendly bureaucracy slow thing,0.7096,Very Happy,0.0000,Neutral,0.7096,Very Happy
1,startup_1,Former Employee,food food food cafe main campus mtv alone mini...,work/life balance balance perk benefit illusio...,food food food cafe main campus mtv alone mini...,0.9806,Very Happy,0.0054,Neutral,0.9807,Very Happy
2,startup_1,Current Employee,software engineer among king hill google engin...,become large come grow pain bureaucracy slow r...,software engineer among king hill google engin...,0.9971,Very Happy,-0.9325,Unhappy,0.9956,Very Happy


# TFIDF 

In [5]:
features_train = df_train['clean_positives']
labels_train = df_train['positive_label']

In [6]:
vectorizer = TfidfVectorizer()
positive_vectorizer = vectorizer.fit_transform(features_train.values.astype('U')) # to convert to unicode

# Dividing Data into Training and Test Sets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(positive_vectorizer, labels_train, test_size=0.20, random_state=101)